In [12]:
words = open('names.txt').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [13]:
# identify unique chars and map to their codes
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# init tensors for 3d
import torch
N = torch.zeros((27, 27, 27), dtype=torch.int32)

# process words w/ hallucinated start/end into tensors
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        N[stoi[ch1], stoi[ch2], stoi[ch3]] += 1

In [14]:
# (attempt to) visualize

import matplotlib.pyplot as plt
%matplotlib inline

fig, axs = plt.subplots(7, 4, figsize=(100, 100))
for i in range(27):
    axs[i//4, i%4].imshow(N[i])
fig.tight_layout()
plt.show()

In [15]:
# precompute probabilities for every trio
P = (N+1).float()
P /= P.sum(2, keepdims=True)
P.shape

torch.Size([27, 27, 27])

In [16]:
P[0][1]

tensor([0.0002, 0.0469, 0.0430, 0.0072, 0.0827, 0.0126, 0.0050, 0.0041, 0.0207,
        0.0349, 0.0063, 0.0171, 0.1427, 0.0868, 0.1406, 0.0025, 0.0041, 0.0023,
        0.1089, 0.0439, 0.0165, 0.0345, 0.0550, 0.0016, 0.0063, 0.0392, 0.0345])

In [17]:
# use pytorch for sampling
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
        p = P[ix1][ix2]
        ix1 = ix2
        ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g)[0].item()
        out.append(itos[ix2])

        if ix2 == 0:
            break
    print(''.join(out))

quia.
yu.
quinslyntien.
nolliahi.
ha.


In [18]:
# write loss function which maximizes for likelihood
log_likelihood = 0.0
n = 0
for w in words:
  shouldprint = w == "daniel"
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1, ix2, ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    if shouldprint:
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')
nll = -log_likelihood

print('Log likelihood', log_likelihood)
print('NLL', nll)
print('Avg NLL', nll/n)

.d: 0.4083 -0.8958
da: 0.1564 -1.8554
an: 0.1288 -2.0493
ni: 0.1279 -2.0569
ie: 0.2357 -1.4451
el: 0.2223 -1.5038
.d: 0.4083 -0.8958
da: 0.1564 -1.8554
an: 0.1288 -2.0493
ni: 0.1279 -2.0569
ie: 0.2357 -1.4451
el: 0.2223 -1.5038
Log likelihood tensor(-410414.9688)
NLL tensor(410414.9688)
Avg NLL tensor(2.0927)


In [97]:
# nn approach
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append(tuple([ix1, ix2]))
        ys.append(ix3)

pairs = list(set(xs))

In [105]:
# init
import torch.nn.functional as F
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27 * 27, 27), generator=g, requires_grad=True)

In [106]:
for k in range(100):
    # fwd
    dxs = torch.tensor([pairs.index(p) for p in xs])
    xenc = F.one_hot(dxs, num_classes=27*27).float()

    logits = xenc @ W # log-counts, or "l-oh-jits"
    counts = logits.exp() # equiv N
    probs = counts / counts.sum(1, keepdims=True)

    loss = -probs[torch.arange(len(xs)), ys].log().mean()
    print(loss.item())

    # bwd
    W.grad = None # 0
    loss.backward()

    # update
    W.data += -50 * W.grad

3.7462050914764404
3.66563081741333
3.5900914669036865
3.519482374191284
3.453794479370117
3.393017292022705
3.337064504623413
3.2857303619384766
3.2386908531188965
3.195549488067627
3.155895709991455
3.119349956512451
3.085581064224243
3.054306983947754
3.0252861976623535
2.998305082321167
2.973172903060913
2.9497170448303223
2.927781105041504
2.9072206020355225
2.887906789779663
2.869722366333008
2.852562665939331
2.836334466934204
2.8209540843963623
2.806347608566284
2.792449951171875
2.7792022228240967
2.766552209854126
2.7544543743133545
2.742866277694702
2.7317519187927246
2.721076726913452
2.7108101844787598
2.7009243965148926
2.6913957595825195
2.6822004318237305
2.6733181476593018
2.6647303104400635
2.6564197540283203
2.648369789123535
2.6405675411224365
2.632998466491699
2.6256518363952637
2.6185152530670166
2.611579179763794
2.6048336029052734
2.5982699394226074
2.5918803215026855
2.5856568813323975
2.579592227935791
2.5736801624298096
2.5679142475128174
2.562288761138916
2.